<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/squareyards_scraping_New_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Import Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import re

from dateutil.relativedelta import relativedelta #
from functools import reduce

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import gspread
from google.colab import auth
auth.authenticate_user() #

from oauth2client.client import GoogleCredentials #
#gc = gspread.authorize(GoogleCredentials.get_application_default()) #

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe
from geopy.distance import geodesic as GD

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def cleanText(row):
  return row.str.upper().str.strip()

def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")

  for (root, dirs, files) in os.walk(file_path):
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()

        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
          temp = pd.read_csv(path,parse_dates=True)
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          #print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.9 MB/s eta 0:00:00


In [ ]:
daysDelta =0

DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')

Last30DaysDate = (datetime.today() - timedelta(days = 30 + daysDelta)).date()
Last60DaysDate = (datetime.today() - timedelta(days = 60 + daysDelta)).date()
Last90DaysDate = (datetime.today() - timedelta(days = 90 + daysDelta)).date()
Last7DaysDate = (datetime.today() - timedelta(days = 7 + daysDelta)).date()
Last45DaysDate = (datetime.today() - timedelta(days = 45 + daysDelta)).date()
MonthStartDate = datetime.today().date().replace(day=1)

start_date = Last30DaysDate.strftime('%d %B %Y')

file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Feb 2023/"

In [ ]:
#get data from spreadsheetID and Range Name function
def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

In [ ]:
# import requests
# import time
# from bs4 import BeautifulSoup
# import json

# base_url = "https://www.squareyards.com/new-projects-in-bangalore?page="
# total_pages = 10  # Adjust the total number of pages you want to scrape

# for page in range(1, total_pages + 1):
#     url = base_url + str(page)
#     response = requests.get(url)
#     content = response.content

#     # Simulate scrolling to load all content
#     scroll_pause_time = 1.0  # Adjust the pause time as needed
#     scrolls = 5  # Adjust the number of scrolls as needed

#     # Scroll to the bottom of the page to load more content
#     for _ in range(scrolls):
#         time.sleep(scroll_pause_time)

#     # Parse the HTML content using BeautifulSoup
#     soup = BeautifulSoup(content, 'html.parser')

#     tile_figure = soup.find('div', class_='tileFigure')
#     project_name_element = tile_figure.find('h2', class_=lambda x: x and 'projectName' in x.split())
#     project_name = project_name_element.text.strip() if project_name_element else "N/A"
#     location = tile_figure.find('span', class_='locationName').text.strip()

#     # Extract price details
#     price_box = soup.find('div', class_='priceWithStar')
#     price_range = price_box.find('div', class_='price DSE_NewProjects_D19').text.strip()

#     # Extract unit details
#     configurations_box = soup.find('div', class_='shortInfoSec')
#     configurations = configurations_box.find_all('li')
#     units = [config.text.strip() for config in configurations]

#     # Extract project information
#     description_box = soup.find('div', class_='descrption')
#     description = description_box.find('div', class_='descrption').text.strip()

#     # Print the extracted information
#     print("Project Name:", project_name)
#     print("Location:", location)
#     print("Price Range:", price_range)
#     print("Units:", units)

#     # Parse the HTML content using BeautifulSoup
#     soup = BeautifulSoup(content, 'html.parser')
#     print("Description:", description)


#**Scraping**


###**BLR**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


base_url = 'https://www.squareyards.com/new-projects-in-bangalore?page='



# Define the range of pages you want to scrape
start_page = 0
end_page = 680
# df = pd.DataFrame(columns = ['Project Name','Location','Price','Units','Size in Acres'])
data = []
for page in range(start_page, end_page + 1):
    # Create the URL for the current page
    url = base_url + str(page)

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the HTML content
        # html_content = response.text

        # Create a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(response.content, 'html.parser')
        tile_boxes = soup.find_all("div", class_="tileBox")
        i=0
        # Extract information from each tile box
        for tile_box in tile_boxes:
            i=i+1
            print(i)
            project_name_element = tile_box.find("h2", class_="projectName")
            project_name = project_name_element.text.strip() if project_name_element else ""

            location_element = tile_box.find("span", class_="locationName")
            location = location_element.text.strip() if location_element else ""

            price_element = tile_box.find("div", class_="price DSE_NewProjects_D19")
            price = price_element.text.strip() if price_element else ""
            # print(tile_box)
            size_container = soup.find('div', class_='shortInfoSec')
            print("Project Name:", project_name)
            print("Location:", location)
            print("Price:", price)

            if size_container:
                    units_element = size_container.find('ul', class_='value').find_all('li')

                    if len(units_element) >= 2:
                        units = units_element[0].text.strip()
                        acres = units_element[1].text.strip()

                    else:
                        units = None
                        acres = None


                        print('Project Name:', project_name)
                        print('Units:', units)
                        print('Size in Acres:', acres)
            data.append({
                            'Project Name': project_name,
                            'Location':location,
                            'Price':price,
                            'Units': units,
                            'Size in Acres': acres
                        })


                # Print the extracted details
                # print('Units:', units)
                # print('Size in Acres:', acres)
            print("---------------------")

df = pd.DataFrame(data)

1
Project Name: Provident Ecopolitan
Location: Bagaluru, Bangalore
Price: ₹ 41.90 Lac - 94.03 Lac
---------------------
2
Project Name: Mana Verdant
Location: Jakkur, Bangalore
Price: ₹ 1.05 Cr - 1.54 Cr
---------------------
3
Project Name: Godrej Ananda
Location: Bagaluru, Bangalore
Price: ₹ 29.99 Lac - 90.55 Lac
---------------------
4
Project Name: Bhartiya City Nikoo Homes 5
Location: Thanisandra Main Road, Bangalore
Price: ₹ 48.00 Lac - 2.55 Cr
---------------------
5
Project Name: Sowparnika Indradhanush
Location: Hoskote, Bangalore
Price: ₹ 30.16 Lac - 66.01 Lac
---------------------
6
Project Name: Provident Park Square
Location: Kanakapura Road, Bangalore
Price: ₹ 49.59 Lac - 1.26 Cr
---------------------
7
Project Name: Sobha Neopolis
Location: Varthur, Bangalore
Price: ₹ 89.00 Lac - 3.58 Cr
---------------------
8
Project Name: Provident Sunworth
Location: Mysore Road, Bangalore
Price: ₹ 57.99 Lac - 71.06 Lac
---------------------
9
Project Name: Provident Capella
Location:

KeyboardInterrupt: ignored

In [ ]:
df

,Project Name,Location,Price,Units,Size in Acres
0,Provident Ecopolitan,"Bagaluru, Bangalore",₹ 41.90 Lac - 94.03 Lac,956 units,10 Acres
1,Mana Verdant,"Jakkur, Bangalore",₹ 1.05 Cr - 1.54 Cr,956 units,10 Acres
2,Godrej Ananda,"Bagaluru, Bangalore",₹ 29.99 Lac - 90.55 Lac,956 units,10 Acres
3,Bhartiya City Nikoo Homes 5,"Thanisandra Main Road, Bangalore",₹ 48.00 Lac - 2.55 Cr,956 units,10 Acres
4,Sowparnika Indradhanush,"Hoskote, Bangalore",₹ 30.16 Lac - 66.01 Lac,956 units,10 Acres
...,...,...,...,...,...
24503,Gowri Ideal Homes,"Bommasandra, Bangalore",Price on request,3 BHK,
24504,Golden Gate Home III,"Attibele, Bangalore",Price on request,3 BHK,
24505,Kumari Sri Nilayam,"Hebbal, Bangalore",Price on request,3 BHK,
24506,Kristal Campus A,"Sarjapur, Bangalore",Price on request,3 BHK,


In [ ]:
sheet  = gc.open_by_url("https://docs.google.com/spreadsheets/d/1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM/edit#gid=0")
collated_data =  pd.DataFrame(sheet.worksheet("BLR").get_all_records())

new_df= collated_data.append(df)
spreadSheetId = "1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName = "BLR"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,new_df)

<ipython-input-111-db2bb4d7f6a6>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df= collated_data.append(df)


###**Chennai**

In [ ]:
base_url = 'https://www.squareyards.com/new-projects-in-chennai?page='



# Define the range of pages you want to scrape
start_page = 0
end_page = 600
# df = pd.DataFrame(columns = ['Project Name','Location','Price','Units','Size in Acres'])
data = []
for page in range(start_page, end_page + 1):
    # Create the URL for the current page
    url = base_url + str(page)

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the HTML content
        # html_content = response.text

        # Create a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(response.content, 'html.parser')
        tile_boxes = soup.find_all("div", class_="tileBox")
        i=0
        # Extract information from each tile box
        for tile_box in tile_boxes:
            i=i+1
            print(i)
            project_name_element = tile_box.find("h2", class_="projectName")
            project_name = project_name_element.text.strip() if project_name_element else ""

            location_element = tile_box.find("span", class_="locationName")
            location = location_element.text.strip() if location_element else ""

            price_element = tile_box.find("div", class_="price DSE_NewProjects_D19")
            price = price_element.text.strip() if price_element else ""
            # print(tile_box)
            size_container = soup.find('div', class_='shortInfoSec')
            print("Project Name:", project_name)
            print("Location:", location)
            print("Price:", price)

            if size_container:
                    units_element = size_container.find('ul', class_='value').find_all('li')

                    if len(units_element) >= 2:
                        units = units_element[0].text.strip()
                        acres = units_element[1].text.strip()

                    else:
                        units = None
                        acres = None


                        print('Project Name:', project_name)
                        print('Units:', units)
                        print('Size in Acres:', acres)
            data.append({
                            'Project Name': project_name,
                            'Location':location,
                            'Price':price,
                            'Units': units,
                            'Size in Acres': acres
                        })


                # Print the extracted details
                # print('Units:', units)
                # print('Size in Acres:', acres)
            print("---------------------")

df = pd.DataFrame(data)

Streaming output truncated to the last 5000 lines.
33
Project Name: Red Sun Service Apartment
Location: Pallikaranai, Chennai
Price: Price on request
---------------------
34
Project Name: Red Rose Madipakkam
Location: Madipakkam, Chennai
Price: Price on request
---------------------
35
Project Name: Real Value Yaduttam
Location: Ashok Nagar, Chennai
Price: Price on request
---------------------
36
Project Name: Real Value Wamika
Location: Kolapakkam, Chennai
Price: Price on request
---------------------
1
Project Name: Real Value Srivari Apartment
Location: Saidapet, Chennai
Price: Price on request
---------------------
2
Project Name: Real Value Sai Villa
Location: Oragadam, Chennai
Price: Price on request
---------------------
3
Project Name: Real Value Sai Uptown
Location: Oragadam, Chennai
Price: Price on request
---------------------
4
Project Name: Real Value Sai Surya
Location: Pallikaranai, Chennai
Price: Price on request
---------------------
5
Project Name: Real Value Sai Sh

In [ ]:
spreadSheetId = "1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName = "Chennai"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,df)

In [ ]:
df.shape

###**Mumbai**

In [ ]:
base_url = 'https://www.squareyards.com/new-projects-in-mumbai?page='



# Define the range of pages you want to scrape
start_page = 0
end_page = 1025
# df = pd.DataFrame(columns = ['Project Name','Location','Price','Units','Size in Acres'])
data = []
for page in range(start_page, end_page + 1):
    # Create the URL for the current page
    url = base_url + str(page)

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the HTML content
        # html_content = response.text

        # Create a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(response.content, 'html.parser')
        tile_boxes = soup.find_all("div", class_="tileBox")
        i=0
        # Extract information from each tile box
        for tile_box in tile_boxes:
            i=i+1
            print(i)
            project_name_element = tile_box.find("h2", class_="projectName")
            project_name = project_name_element.text.strip() if project_name_element else ""

            location_element = tile_box.find("span", class_="locationName")
            location = location_element.text.strip() if location_element else ""

            price_element = tile_box.find("div", class_="price DSE_NewProjects_D19")
            price = price_element.text.strip() if price_element else ""
            # print(tile_box)
            size_container = soup.find('div', class_='shortInfoSec')
            print("Project Name:", project_name)
            print("Location:", location)
            print("Price:", price)

            if size_container:
                    units_element = size_container.find('ul', class_='value').find_all('li')

                    if len(units_element) >= 2:
                        units = units_element[0].text.strip()
                        acres = units_element[1].text.strip()

                    else:
                        units = None
                        acres = None


                        print('Project Name:', project_name)
                        print('Units:', units)
                        print('Size in Acres:', acres)
            data.append({
                            'Project Name': project_name,
                            'Location':location,
                            'Price':price,
                            'Units': units,
                            'Size in Acres': acres
                        })


                # Print the extracted details
                # print('Units:', units)
                # print('Size in Acres:', acres)
            print("---------------------")

df = pd.DataFrame(data)

Streaming output truncated to the last 5000 lines.
Price: Price on request
---------------------
31
Project Name: Somraj Building
Location: Kalbadevi, Mumbai
Price: Price on request
---------------------
32
Project Name: Somnath Building
Location: Kalbadevi, Mumbai
Price: Price on request
---------------------
33
Project Name: Someshwar Niwas CHS
Location: Dadar West, Mumbai
Price: Price on request
---------------------
34
Project Name: Somaiya House
Location: Vile Parle East, Mumbai
Price: Price on request
---------------------
35
Project Name: Somabai Building
Location: Bhuleshwar, Mumbai
Price: Price on request
---------------------
36
Project Name: Soma Heights
Location: Vasai East, Mumbai
Price: Price on request
---------------------
1
Project Name: Som Niwas
Location: Dadar East, Mumbai
Price: Price on request
Project Name: Som Niwas
Units: None
Size in Acres: None
---------------------
2
Project Name: Som Dhan Apartment
Location: Bandra West, Mumbai
Price: Price on request
Proje

In [ ]:
df.shape

(36936, 5)

In [ ]:
spreadSheetId = "1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName = "Mumbai"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,df)

###**Navi Mumbai**

In [ ]:
base_url = 'https://www.squareyards.com/new-projects-in-thane?page='



# Define the range of pages you want to scrape
start_page = 0
end_page = 349
# df = pd.DataFrame(columns = ['Project Name','Location','Price','Units','Size in Acres'])
data = []
for page in range(start_page, end_page + 1):
    # Create the URL for the current page
    url = base_url + str(page)

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the HTML content
        # html_content = response.text

        # Create a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(response.content, 'html.parser')
        tile_boxes = soup.find_all("div", class_="tileBox")
        i=0
        # Extract information from each tile box
        for tile_box in tile_boxes:
            i=i+1
            print(i)
            project_name_element = tile_box.find("h2", class_="projectName")
            project_name = project_name_element.text.strip() if project_name_element else ""

            location_element = tile_box.find("span", class_="locationName")
            location = location_element.text.strip() if location_element else ""

            price_element = tile_box.find("div", class_="price DSE_NewProjects_D19")
            price = price_element.text.strip() if price_element else ""
            # print(tile_box)
            size_container = soup.find('div', class_='shortInfoSec')
            print("Project Name:", project_name)
            print("Location:", location)
            print("Price:", price)

            if size_container:
                    units_element = size_container.find('ul', class_='value').find_all('li')

                    if len(units_element) >= 2:
                        units = units_element[0].text.strip()
                        acres = units_element[1].text.strip()

                    else:
                        units = None
                        acres = None


                        print('Project Name:', project_name)
                        print('Units:', units)
                        print('Size in Acres:', acres)
            data.append({
                            'Project Name': project_name,
                            'Location':location,
                            'Price':price,
                            'Units': units,
                            'Size in Acres': acres
                        })


                # Print the extracted details
                # print('Units:', units)
                # print('Size in Acres:', acres)
            print("---------------------")

df = pd.DataFrame(data)

Streaming output truncated to the last 5000 lines.
Project Name: Purushottam Park
Units: None
Size in Acres: None
---------------------
2
Project Name: Purshottam Nayan
Location: Ghodbunder Road, Thane
Price: Price on request
Project Name: Purshottam Nayan
Units: None
Size in Acres: None
---------------------
3
Project Name: Purshotam Sadan Apartment
Location: Kolshet, Thane
Price: Price on request
Project Name: Purshotam Sadan Apartment
Units: None
Size in Acres: None
---------------------
4
Project Name: Purnashanti Heights
Location: Kolshet, Thane
Price: Price on request
Project Name: Purnashanti Heights
Units: None
Size in Acres: None
---------------------
5
Project Name: PuranikCity Building No D1 D2 & H Chs Ltd
Location: Ghodbunder Road, Thane
Price: Price on request
Project Name: PuranikCity Building No D1 D2 & H Chs Ltd
Units: None
Size in Acres: None
---------------------
6
Project Name: Puranik Snehabandha Sahaniwas
Location: Kasarvadavali, Thane
Price: Price on request
Proje

In [ ]:
df.shape

(12600, 5)

In [ ]:
spreadSheetId = "1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName = "thane"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,df)

###**City**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


base_url = 'https://www.squareyards.com/new-projects-in-goa?page='

# Define the range of pages you want to scrape
start_page = 0
end_page = 100
# df = pd.DataFrame(columns = ['Project Name','Location','Price','Units','Size in Acres'])
data = []
for page in range(start_page, end_page + 1):
    # Create the URL for the current page
    url = base_url + str(page)

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the HTML content
        # html_content = response.text

        # Create a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(response.content, 'html.parser')
        tile_boxes = soup.find_all("div", class_="tileBox")
        i=0
        # Extract information from each tile box
        for tile_box in tile_boxes:
            i=i+1
            print(i)
            project_name_element = tile_box.find("h2", class_="projectName")
            project_name = project_name_element.text.strip() if project_name_element else ""

            location_element = tile_box.find("span", class_="locationName")
            location = location_element.text.strip() if location_element else ""

            price_element = tile_box.find("div", class_="price DSE_NewProjects_D19")
            price = price_element.text.strip() if price_element else ""
            # print(tile_box)
            size_container = soup.find('div', class_='shortInfoSec')
            print("Project Name:", project_name)
            print("Location:", location)
            print("Price:", price)

            if size_container:
                    units_element = size_container.find('ul', class_='value').find_all('li')

                    if len(units_element) >= 2:
                        units = units_element[0].text.strip()
                        acres = units_element[1].text.strip()

                    else:
                        units = None
                        acres = None


                        print('Project Name:', project_name)
                        print('Units:', units)
                        print('Size in Acres:', acres)
            data.append({
                            'Project Name': project_name,
                            'Location':location,
                            'Price':price,
                            'Units': units,
                            'Size in Acres': acres
                        })


                # Print the extracted details
                # print('Units:', units)
                # print('Size in Acres:', acres)
            print("---------------------")

df = pd.DataFrame(data)



spreadSheetId = "1PkvBB2SOOKCOILINVuYM90RVnZ2ARy7LLvWSzs9nRMA"
rangeName = "Goa"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()
set_with_dataframe(ws,df)

Streaming output truncated to the last 5000 lines.
24
Project Name: Saldanha Palms II
Location: Candolim, Goa
Price: ₹ 40.33 Lac - 66.26 Lac
Project Name: Saldanha Palms II
Units: None
Size in Acres: None
---------------------
25
Project Name: Akar Suburbia
Location: Cuncolim, Goa
Price: ₹ 72.63 Lac - 1.19 Cr
Project Name: Akar Suburbia
Units: None
Size in Acres: None
---------------------
26
Project Name: Pruthvi Privada
Location: Calangute, Goa
Price: ₹ 41.59 Lac - 98.15 Lac
Project Name: Pruthvi Privada
Units: None
Size in Acres: None
---------------------
27
Project Name: Talak Lotus Park
Location: Mapusa, Goa
Price: 58.25 Lac
Project Name: Talak Lotus Park
Units: None
Size in Acres: None
---------------------
28
Project Name: Nitin Corlim Gardens
Location: Corlim, Goa
Price: ₹ 43.98 Lac - 47.12 Lac
Project Name: Nitin Corlim Gardens
Units: None
Size in Acres: None
---------------------
29
Project Name: CD Diva Gogol
Location: Margao, Goa
Price: ₹ 64.12 Lac - 88.93 Lac
Project Name

###**City 2**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


base_url = 'https://www.squareyards.com/new-projects-in-surat?page='

# Define the range of pages you want to scrape
start_page = 0
end_page = 30
# df = pd.DataFrame(columns = ['Project Name','Location','Price','Units','Size in Acres'])
data = []
for page in range(start_page, end_page + 1):
    # Create the URL for the current page
    url = base_url + str(page)

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the HTML content
        # html_content = response.text

        # Create a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(response.content, 'html.parser')
        tile_boxes = soup.find_all("div", class_="tileBox")
        i=0
        # Extract information from each tile box
        for tile_box in tile_boxes:
            i=i+1
            print(i)
            project_name_element = tile_box.find("h2", class_="projectName")
            project_name = project_name_element.text.strip() if project_name_element else ""

            location_element = tile_box.find("span", class_="locationName")
            location = location_element.text.strip() if location_element else ""

            price_element = tile_box.find("div", class_="price DSE_NewProjects_D19")
            price = price_element.text.strip() if price_element else ""
            # print(tile_box)
            size_container = soup.find('div', class_='shortInfoSec')
            print("Project Name:", project_name)
            print("Location:", location)
            print("Price:", price)

            if size_container:
                    units_element = size_container.find('ul', class_='value').find_all('li')

                    if len(units_element) >= 2:
                        units = units_element[0].text.strip()
                        acres = units_element[1].text.strip()

                    else:
                        units = None
                        acres = None


                        print('Project Name:', project_name)
                        print('Units:', units)
                        print('Size in Acres:', acres)
            data.append({
                            'Project Name': project_name,
                            'Location':location,
                            'Price':price,
                            'Units': units,
                            'Size in Acres': acres
                        })


                # Print the extracted details
                # print('Units:', units)
                # print('Size in Acres:', acres)
            print("---------------------")

df = pd.DataFrame(data)



spreadSheetId = "1PkvBB2SOOKCOILINVuYM90RVnZ2ARy7LLvWSzs9nRMA"
rangeName = "Surat"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()
set_with_dataframe(ws,df)

In [ ]:
spreadSheetId = "1PkvBB2SOOKCOILINVuYM90RVnZ2ARy7LLvWSzs9nRMA"
rangeName = "Patna"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,df)

In [ ]:
sheet  = gc.open_by_url("https://docs.google.com/spreadsheets/d/1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM/edit#gid=0")
collated_data =  pd.DataFrame(sheet.worksheet("Emerging").get_all_records())
new_df= collated_data.append(df)



spreadSheetId = "1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName = "Emerging"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,new_df)

PermissionError: 

###**Merge all cities**

In [ ]:
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

In [ ]:

spreadSheetId="1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName="HYD"
d1=get_data_df(spreadSheetId, rangeName)

rangeName="BLR"
d2=get_data_df(spreadSheetId, rangeName)

rangeName="Chennai"
d3=get_data_df(spreadSheetId, rangeName)

rangeName="Mumbai"
d4=get_data_df(spreadSheetId, rangeName)

rangeName="Navi Mumbai"
d5=get_data_df(spreadSheetId, rangeName)

rangeName="thane"
d6=get_data_df(spreadSheetId, rangeName)

rangeName="Pune"
d7=get_data_df(spreadSheetId, rangeName)

rangeName="Noida"
d8=get_data_df(spreadSheetId, rangeName)

rangeName="greater-noida"
d9=get_data_df(spreadSheetId, rangeName)

rangeName="Ahmd"
d10=get_data_df(spreadSheetId, rangeName)

rangeName="Gurgaon"
d11=get_data_df(spreadSheetId, rangeName)

rangeName="Emerging"
d12=get_data_df(spreadSheetId, rangeName)


In [ ]:
city_merge = pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12])

In [ ]:
city_merge

,Project Name,Location,Price,Units,Size in Acres
0,The Prestige City Hyderabad,"Rajendra Nagar, Hyderabad",₹ 76.64 Lac - 2.23 Cr,4647 units,64 Acres
1,Candeur Lakescape,"Kondapur, Hyderabad",₹ 1.02 Cr - 1.61 Cr,4647 units,64 Acres
2,ASBL Loft,"Financial District, Hyderabad",₹ 1.53 Cr - 1.68 Cr,4647 units,64 Acres
3,Phoenix Trivium,"Hafeezpet, Hyderabad",Price on request,4647 units,64 Acres
4,SAS Diamond Towers,"Financial District, Hyderabad",3.55 Cr,4647 units,64 Acres
...,...,...,...,...,...
8160,Green Golden Beach Residences,"Bhogapuram, Vizag",₹ 12.59 Lac - 16.79 Lac,1025 units,14 Acres
8161,Shriram Celebrity Towers,"Madhurawada, Vizag",₹ 48.41 Lac - 79.88 Lac,1025 units,14 Acres
8162,Shriram Panorama Hills Paramount Towers,"Madhurawada, Vizag",₹ 79.65 Lac - 1.37 Cr,1025 units,14 Acres
8163,Panorama Hills Daffodils Serene Villas,"Madhurawada, Vizag",1.50 Cr,1025 units,14 Acres


In [ ]:
spreadSheetId = "1NWAJRbNlxg2jhqh4Yo_2VIp6PNYBVSODXtlG2JgzlPM"
rangeName = "New Projects"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,city_merge)